# Loading Packages and Libraries

In [34]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import metrics
import seaborn as sns

from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from google.colab import auth
from google.cloud import bigquery
import os
auth.authenticate_user()
print('Authenticated')

Authenticated


# Connect to project in big query

In [35]:
#large datasets
%load_ext google.colab.data_table
#traditional 
#%unload_ext google.colab.data_table
project_id = 'mit-covid-19-datathon'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


# Mount Drive

In [36]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Big Query (Retrieving census data)

In [37]:
client = bigquery.Client(project=project_id)

df_2 = client.query('''
  SELECT * FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr`
where geo_id IN ('10453', '10457', '10460', '10458', '10467', '10468', '10451', '10452', '10456', '10454', '10455', '10459', '10474', 
'10463', '10471', '10466', '10469', '10470', '10475', '10461', '10462', '10464', '10465', '10472', '10473', '11212', '11213', '11216', '11233', '11238', '11209', '11214', '11228', '11204', '11218', '11219', '11230', '11234', '11236', '11239', '11223', '11224', '11229', '11235', '11201', '11205', '11215', '11217', '11231', '11203', '11210', '11225', '11226', 	'11207', '11208', '11211', '11222', '11220', '11232', '11206', '11221', '11237', '10026', '10027', '10030', '10037', '10039', '10001', '10011', '10018', '10019', '10020', '10036', '10029', '10035', '10010', '10016', '10017', '10022', '10012', '10013', '10014', 	'10004', '10005', '10006', '10007', '10038', '10280', 
'10002', '10003', '10009', '10021', '10028', '10044', '10065', '10075', '10128', '10023', '10024', '10025', 	'10031', '10032', '10033', '10034', '10040', '11361', '11362', '11363', '11364', '11354', '11355', '11356', '11357', '11358', '11359', '11360', 	'11365', '11366', '11367', 	'11412', '11423', '11432', '11433', '11434', '11435', '11436', 	'11101', '11102', '11103', '11104', '11105', '11106', 
'11374', '11375', '11379', '11385', 	'11691', '11692', '11693', '11694', '11695', '11697', 	'11004', '11005', '11411', '11413', '11422', '11426', '11427', '11428', '11429', 	'11414', '11415', '11416', '11417', '11418', '11419', '11420', '11421', 	'11368', '11369', '11370', '11372', '11373', '11377', '11378', 	'10302', '10303', '10310',	'10306', '10307', '10308', '10309', '10312', '10301', '10304', '10305', '10314')
''' ).to_dataframe()

print('Full dataset has been loaded :-)')

Full dataset has been loaded :-)


# Dropping a few columns for now

In [0]:
df_2.drop(['do_date', 'speak_spanish_at_home', 'speak_only_english_at_home', 'speak_spanish_at_home_low_english'], axis=1, inplace=True)

In [0]:
pop_dens = pd.read_csv('/content/drive/My Drive/data/pop_density.csv')

In [44]:
pop_dens.isnull().values.any()

False

In [65]:
pop_dens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771 entries, 0 to 1770
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Pop_Density      1771 non-null   float64
 1   geo_id           1771 non-null   object 
 2        Population  1771 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 41.6+ KB


In [0]:
pop_dens['geo_id'] = pop_dens['geo_id'].astype(int)
pop_dens['geo_id'] = pop_dens['geo_id'].astype(str)


In [0]:
df = pd.merge(df_2, pop_dens, on = 'geo_id', how = 'inner')

In [0]:
df_2 = df

In [0]:
df_2.columns = df_2.columns.str.replace(' ', '')


In [82]:
df_2.columns

Index(['geo_id', 'total_pop', 'households', 'male_pop', 'female_pop',
       'median_age', 'male_under_5', 'male_5_to_9', 'male_10_to_14',
       'male_15_to_17',
       ...
       'management_business_sci_arts_employed', 'sales_office_employed',
       'in_grades_1_to_4', 'in_grades_5_to_8', 'in_grades_9_to_12',
       'in_school', 'in_undergrad_college', 'Pop_Density', 'Population',
       'land_area'],
      dtype='object', length=239)

In [61]:
df_2.shape

(176, 239)

In [0]:
df_2["land_area"] = ""


In [0]:
df_2['land_area'] = df_2['Population']/df_2['Pop_Density']


In [0]:
df_2["new_pop_density"] = ""


In [0]:
df_2['new_pop_density'] = df_2['total_pop']/df_2['land_area']


# Imputation

In [55]:
df_2 = df_2.fillna(0)
df_2.isnull().values.any()

False

In [56]:
# Viewing the data in the folder present from the drive
!ls "/content/drive/My Drive/data/COVID_data/"

4_03.csv  4_10.csv  4_16.csv  4_22.csv	4_28.csv  5_04.csv  5_10.csv  5_16.csv
4_04.csv  4_11.csv  4_17.csv  4_23.csv	4_29.csv  5_05.csv  5_11.csv  5_17.csv
4_05.csv  4_12.csv  4_18.csv  4_24.csv	4_30.csv  5_06.csv  5_12.csv  5_18.csv
4_07.csv  4_13.csv  4_19.csv  4_25.csv	5_01.csv  5_07.csv  5_13.csv  5_19.csv
4_08.csv  4_14.csv  4_20.csv  4_26.csv	5_02.csv  5_08.csv  5_14.csv
4_09.csv  4_15.csv  4_21.csv  4_27.csv	5_03.csv  5_09.csv  5_15.csv


# find the csv files and Sort inputs in chronological Order

In [0]:
data_files = glob.glob("/content/drive/My Drive/data/COVID_data/*.csv")

In [0]:
data_files = sorted(data_files)[::-1]

In [0]:
# Compute average daily cases increase
daily_infection_rate = pd.DataFrame()
df_csv = pd.read_csv(data_files[0], index_col=None, header=0)

daily_infection_rate['MODZCTA'] = df_csv['MODZCTA']
daily_infection_rate['Positive_ref'] = df_csv['Positive']
daily_infection_rate['IR'] = np.nan
# for filename in data_files[1:]:
#     df_csv = pd.read_csv(filename, index_col=None, header=0)
#     #df_csv.rename(columns={'MODZCTA':'geo_id'}, inplace=True)

#     daily_infection_rate = pd.merge(daily_infection_rate, df_csv[['MODZCTA', 'Positive']], on = 'MODZCTA', how = 'inner')
#     daily_infection_rate[filename.split('/')[-1]] = daily_infection_rate['Positive_ref'] - daily_infection_rate['Positive']
#     daily_infection_rate['IR'] = daily_infection_rate[[filename.split('/')[-1],'IR']].mean(axis = 1)
#     daily_infection_rate['Positive_ref'] = daily_infection_rate['Positive']
    
#     daily_infection_rate = daily_infection_rate.drop(columns=['Positive'])
for filename in data_files[1:]:
    df_csv = pd.read_csv(filename, index_col=None, header=0)
    #df_csv.rename(columns={'MODZCTA':'geo_id'}, inplace=True)
    daily_infection_rate = pd.merge(daily_infection_rate, df_csv[['MODZCTA', 'Positive']], on = 'MODZCTA', how = 'inner')
    daily_infection_rate[filename.split('/')[-1]] = (daily_infection_rate['Positive_ref'] - daily_infection_rate['Positive'])*100/daily_infection_rate['Positive']
    daily_infection_rate['IR'] = daily_infection_rate[[filename.split('/')[-1],'IR']].mean(axis = 1)
    daily_infection_rate['Positive_ref'] = daily_infection_rate['Positive']
    daily_infection_rate = daily_infection_rate.drop(columns=['Positive'])   

In [0]:
daily_infection_rate.rename(columns={'MODZCTA':'geo_id'}, inplace=True)
daily_infection_rate = daily_infection_rate.iloc[1:]
daily_infection_rate['geo_id'] = daily_infection_rate['geo_id'].astype(int)
#daily_infection_rate.info()
daily_infection_rate['geo_id'] = daily_infection_rate['geo_id'].astype(str)

In [0]:
IR_Daily_and_Avge = pd.DataFrame()
IR_Daily_and_Avge['geo_id'] = daily_infection_rate['geo_id']
IR_Daily_and_Avge['IR'] = daily_infection_rate['IR']
IR_Daily_and_Avge[daily_infection_rate.columns[3:][::-1]] = pd.DataFrame(daily_infection_rate[daily_infection_rate.columns[3:][::-1]])
IR_Daily_and_Avge.to_csv('IR_Daily_and_Avge.csv',index=False)
#!cp IR_Daily_and_Avge.csv "drive/My Drive/"
daily_infection_rate = pd.DataFrame(IR_Daily_and_Avge[['geo_id','IR']])
#daily_infection_rate.head()

# Merging the census data with the covid cases

In [0]:
census_ir = pd.merge(df_2, daily_infection_rate, on = 'geo_id', how = 'inner')


# Importing mobility data (citistart-citiend (citibike) and subway data)

In [0]:
#importing other two file from citibikestart and end.csv
citistart = pd.read_csv("/content/drive/My Drive/data/march_citistart.csv")
citiend = pd.read_csv("/content/drive/My Drive/data/march_citisend.csv")
subway = pd.read_csv("/content/drive/My Drive/data/subway_updated.csv")

# Pre-Processing (renaming)

In [0]:
#changing the column name from zipcode to geo_id (useful when joing tables :-) )
citistart1 = citistart.rename(columns={'zipcode': 'geo_id'})
citiend1 = citiend.rename(columns={'zipcode': 'geo_id'})
subway = subway.rename(columns={'zipcode': 'geo_id'})
#changing the column name from tripcount to tripcount_end 
citiend1 = citiend1.rename(columns={'tripcount': 'tripcount_end'})

# Pre-Processing (dropping)

In [0]:
#drop the first column
citistart1.drop(['Unnamed: 0'], axis=1, inplace=True)
citiend1.drop(['Unnamed: 0'], axis=1, inplace=True)
subway.drop(['Unnamed: 0'], axis=1, inplace=True)

# Pre-Processing(datatype conversion)

In [0]:
#changing geo_id datatype (important when joining with the census data)
#for citistar1 dataframe
citistart1['geo_id'] = citistart1['geo_id'].astype(int)
citistart1['geo_id'] = citistart1['geo_id'].astype(str)
#for citiend1 dataframe
citiend1['geo_id'] = citiend1['geo_id'].astype(int)
citiend1['geo_id'] = citiend1['geo_id'].astype(str)
#for subway dataframe
subway['geo_id'] = subway['geo_id'].astype(int)
subway['geo_id'] = subway['geo_id'].astype(str)

# citiend1.info()
#citistart1.info()

# Merging Datasets

In [0]:
df_merge = pd.merge(census_ir, citistart1, on = 'geo_id', how = 'left')

In [0]:
df_merge1 = pd.merge(df_merge, citiend1, on = 'geo_id', how = 'left')


# Filling NANs with zeros

In [0]:
df_merge1= df_merge1.fillna(0)

# Merging with subway

In [0]:
sub_merge = pd.merge(df_merge1, subway, on = 'geo_id', how = 'left')

# Filling NANs with zeros

In [0]:
sub_merge= sub_merge.fillna(0)

In [0]:
sub_merge_final = sub_merge.drop(sub_merge.index[174])

# Saving to a csv file

In [0]:
sub_merge_final.to_csv(r'/content/drive/My Drive/data/final_feature.csv', index=False) 
